In [1]:
import keras.backend as K
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Activation, Dropout, Input
from keras.models import Sequential, Model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers.core import Lambda
from keras.engine.topology import Layer, InputSpec

import os
from os import listdir, mkdir
from scipy.misc import imread
import numpy as np
from sklearn.utils import shuffle
from scipy.io import loadmat
import h5py
import sys

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
class LocalResponseNormalization(Layer):

    def __init__(self, n=5, alpha=0.0005, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x, mask=None):
        if K.image_dim_ordering == "th":
            _, f, r, c = self.shape
        else:
            _, r, c, f = self.shape
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1, 1),
            padding="same", pool_mode="avg")
        if K.image_dim_ordering == "th":
            summed = K.sum(pooled, axis=1, keepdims=True)
            averaged = self.alpha * K.repeat_elements(summed, f, axis=1)
        else:
            summed = K.sum(pooled, axis=3, keepdims=True)
            averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom

    def get_output_shape_for(self, input_shape):
        return input_shape
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [4]:
img_height = 54
img_width = 32
num_channels = 3
num_classes = 2
num_images_train = 208387
num_images_test = 132780+83056

size_set1 = 100000
size_set2 = 100000
size_set3 = 100000

set1 = '../PKLot/PKLotSegmented/PUC/'
set2 = '../PKLot/PKLotSegmented/UFPR04/'
set3 = '../PKLot/PKLotSegmented/UFPR05/'


dataset1 = np.zeros((size_set1, img_height, img_width, num_channels))
dataset2 = np.zeros((size_set2, img_height, img_width, num_channels))
dataset3 = np.zeros((size_set3, img_height, img_width, num_channels))

labels1 = np.zeros(size_set1)
labels2 = np.zeros(size_set2)
labels3 = np.zeros(size_set3)



counter1 = 0
counter2 = 0
counter3 = 0

if not os.path.isdir('weights'):
    mkdir('weights')

if not os.path.isdir(set1):
    print('PKLot dataset folder missing... Exiting!!')
    sys.exit(0)

#------------------------------------------------------------- Preparing Data
print('Preparing data...')

weathers = [w for w in listdir(set1)]
for weather in weathers:
    root_ = set1 + weather + '/'
    print("Inside folder: %s" % root_)
    days = [day for day in listdir(root_)]

    
    # add to train data
    for day in days:
        _root_ = root_ + day + '/'
        print("Inside folder: %s" % _root_)
        labels = [label for label in listdir(_root_)]
        for i, label in enumerate(labels):
            img_root = _root_ + label + '/'
            img_names = [img for img in listdir(img_root)]
            for j,img_name in enumerate(img_names):
                if (j+1) % 1000 == 0:
                    print("Inside folder: %s, image # %d" % (img_root, j+1))
                img = imread(img_root + img_name)
                img = np.resize(img, (img_height, img_width, num_channels))
                dataset1[counter1] = img
                labels1[counter1] = 1-i
                counter1 += 1



Preparing data...
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/


/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Empty/, image # 6000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Rainy/2012-10-11/Occupied/, image # 6000
Inside fol

IndexError: index 100000 is out of bounds for axis 0 with size 100000

In [5]:
weathers = [w for w in listdir(set2)]
for weather in weathers:
    root_ = set2 + weather + '/'
    print("Inside folder: %s" % root_)
    days = [day for day in listdir(root_)]

    
    # add to train data
    for day in days:
        _root_ = root_ + day + '/'
        print("Inside folder: %s" % _root_)
        labels = [label for label in listdir(_root_)]
        for i, label in enumerate(labels):
            img_root = _root_ + label + '/'
            img_names = [img for img in listdir(img_root)]
            for j,img_name in enumerate(img_names):
                if (j+1) % 1000 == 0:
                    print("Inside folder: %s, image # %d" % (img_root, j+1))
                img = imread(img_root + img_name)
                img = np.resize(img, (img_height, img_width, num_channels))
                dataset2[counter2] = img
                labels2[counter2] = 1-i
                counter2 += 1




Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2013-01-17/


/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2012-12-29/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2013-01-18/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2012-12-08/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2012-12-21/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2012-12-07/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2012-12-14/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2012-12-19/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2012-12-26/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2012-12-26/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2013-01-21/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2012-12-15/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2012-12-28/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2012-12-11/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Rainy/2013-01-16/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Sunny/
Inside folder: ../PKLot/PKLotSe

Inside folder: ../PKLot/PKLotSegmented/UFPR04/Cloudy/2012-12-15/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Cloudy/2013-01-22/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Cloudy/2013-01-22/Occupied/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Cloudy/2013-01-22/Occupied/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Cloudy/2012-12-16/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Cloudy/2012-12-28/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Cloudy/2012-12-28/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Cloudy/2012-12-28/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Cloudy/2012-12-28/Empty/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Cloudy/2013-01-16/
Inside folder: ../PKLot/PKLotSegmented/UFPR04/Cloudy/2013-01-16/Empty/, image # 1000


IndexError: index 100000 is out of bounds for axis 0 with size 100000

In [6]:
weathers = [w for w in listdir(set3)]
for weather in weathers:
    root_ = set3 + weather + '/'
    print("Inside folder: %s" % root_)
    days = [day for day in listdir(root_)]

    
    # add to train data
    for day in days:
        _root_ = root_ + day + '/'
        print("Inside folder: %s" % _root_)
        labels = [label for label in listdir(_root_)]
        for i, label in enumerate(labels):
            img_root = _root_ + label + '/'
            img_names = [img for img in listdir(img_root)]
            for j,img_name in enumerate(img_names):
                if (j+1) % 1000 == 0:
                    print("Inside folder: %s, image # %d" % (img_root, j+1))
                img = imread(img_root + img_name)
                img = np.resize(img, (img_height, img_width, num_channels))
                dataset3[counter3] = img
                labels3[counter3] = 1-i
                counter3 += 1


Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-03-19/


/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-04-12/
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-03-20/
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-03-20/Occupied/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-03-05/
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-03-05/Occupied/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-03-13/
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-03-13/Occupied/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-04-13/
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-04-13/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-03-16/
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Rainy/2013-02-26/
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Sunny/
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Sunny/2013-03-18/
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Sunny/2013-03-18/Occupied/, image # 1000
Ins

Inside folder: ../PKLot/PKLotSegmented/UFPR05/Sunny/2013-02-26/
Inside folder: ../PKLot/PKLotSegmented/UFPR05/Sunny/2013-03-22/


IndexError: index 100000 is out of bounds for axis 0 with size 100000

In [17]:
trainX, trainY = shuffle(dataset1, labels1)
validX, validY = shuffle(dataset2, labels2)

In [79]:
def VGG_16():
    '''Model definition'''

    model = Sequential()
    model.add(Conv2D(64, (11, 11,), padding='valid', strides=(4,4), input_shape=(img_height,img_width,num_channels), name='conv1'))
    model.add(Activation('relu', name='relu1'))
    model.add(LocalResponseNormalization(name='norm1'))
    model.add(MaxPooling2D((2,2), padding='same', name='pool1'))

    model.add(Conv2D(256, (5,5), padding='same', name='conv2'))
    model.add(Activation('relu', name='relu2'))
    model.add(LocalResponseNormalization(name='norm2'))
    model.add(MaxPooling2D((2,2), padding='same', name='pool2'))

    model.add(Conv2D(256, (3, 3), padding='same', name='conv3'))
    model.add(Activation('relu', name='relu3'))
    model.add(Conv2D(256, (3, 3), padding='same', name='conv4'))
    model.add(Activation('relu', name='relu4'))
    model.add(Conv2D(256, (3, 3), padding='same', name='conv5'))
    model.add(Activation('relu', name='relu5'))
    model.add(MaxPooling2D((2,2), padding='same', name='pool5'))

    return model

In [80]:
def copy_mat_to_keras(kmodel, weights_path):
    '''Utility function
    Loads weights from .mat file to keras model
    '''
    print('IN COPY MAT TO KERAS')
    kerasnames = [lr.name for lr in kmodel.layers]
    data = loadmat(weights_path, matlab_compatible=False, struct_as_record=False)
    layers = data['layers']
    description = data['meta'][0,0].classes[0,0].description

    prmt = (0,1,2,3)

    for i in range(layers.shape[1]):
        matname = layers[0,i][0,0].name[0]
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            if len(layers[0,i][0,0].weights) > 0:
                l_weights = layers[0,i][0,0].weights[0,0]
                l_bias = layers[0,i][0,0].weights[0,1]
                f_l_weights = l_weights.transpose(prmt)
                assert (f_l_weights.shape == kmodel.layers[kindex].get_weights()[0].shape)
                assert (l_bias.shape[1] == 1)
                assert (l_bias[:,0].shape == kmodel.layers[kindex].get_weights()[1].shape)
                assert (len(kmodel.layers[kindex].get_weights()) == 2)
                kmodel.layers[kindex].set_weights([f_l_weights, l_bias[:,0]])

In [81]:
model = VGG_16()
# Load pre-trained weights from .mat file
#copy_mat_to_keras(model, 'weights/imagenet-vgg-f.mat')
copy_mat_to_keras(model,'imagenet-vgg-f.mat')
# Freeze the convolutional layers
for layer in model.layers:
    layer.trainable = False

model.add(Flatten())
model.add(Dropout(0.1, name='dropout6'))
model.add(Dense(4096, activation='relu', name='fc6'))
model.add(Dropout(0.1, name='dropout7'))
model.add(Dense(4096, activation='relu', name='fc7'))
model.add(Dropout(0.1, name='dropout8'))
model.add(Dense(1, activation='sigmoid', name='predictions'))

model.summary()

IN COPY MAT TO KERAS
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 11, 6, 64)         23296     
_________________________________________________________________
relu1 (Activation)           (None, 11, 6, 64)         0         
_________________________________________________________________
norm1 (LocalResponseNormaliz (None, 11, 6, 64)         0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 6, 3, 64)          0         
_________________________________________________________________
conv2 (Conv2D)               (None, 6, 3, 256)         409856    
_________________________________________________________________
relu2 (Activation)           (None, 6, 3, 256)         0         
_________________________________________________________________
norm2 (LocalResponseNormaliz (None, 6, 3, 256)         

In [82]:
batch_size = 128
epochs = 10
learning_rate = 0.001
weight_decay = 0.00005
nesterov = True
momentum = 0.99

In [83]:
#------------------------------------------------------------- Training

# optimizer
#sgd = SGD(lr=learning_rate, decay=weight_decay, momentum=momentum, nesterov=nesterov)
sgd=Adam(lr=learning_rate,beta_1=0.9,beta_2=0.999, epsilon=None, decay=weight_decay, amsgrad=False)#sgd=Adam(lr=learning_rate,beta_1=0.9,beta_2=0.999, epsilon=None, decay=weight_decay, amsgrad=False)
# Callbacks
# tb = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)
# checkpointer = ModelCheckpoint(filepath="./weights/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5", verbose=1)

model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(validX, validY))


probs = model.predict(trainX, verbose=1)
print('Training complete.')

Train on 100000 samples, validate on 100000 samples
Epoch 1/10
100000/100000 [==============================] - 43s 425us/step - loss: 0.0683 - acc: 0.9834 - val_loss: 0.7353 - val_acc: 0.7353
Epoch 2/10
100000/100000 [==============================] - 41s 407us/step - loss: 0.0309 - acc: 0.9921 - val_loss: 1.0394 - val_acc: 0.7078
Epoch 3/10
100000/100000 [==============================] - 41s 407us/step - loss: 0.0271 - acc: 0.9933 - val_loss: 0.9662 - val_acc: 0.7168
Epoch 4/10
100000/100000 [==============================] - 41s 408us/step - loss: 0.0254 - acc: 0.9939 - val_loss: 1.9784 - val_acc: 0.6629
Epoch 5/10
100000/100000 [==============================] - 41s 406us/step - loss: 0.0228 - acc: 0.9945 - val_loss: 1.7142 - val_acc: 0.6682
Epoch 6/10
100000/100000 [==============================] - 41s 407us/step - loss: 0.0214 - acc: 0.9949 - val_loss: 1.6138 - val_acc: 0.6594
Epoch 7/10
 36992/100000 [==========>...................] - ETA: 19s - loss: 0.0235 - acc: 0.9946

KeyboardInterrupt: 